In [96]:
import pandas as pd
import re
from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [97]:
header_list = ['DATE', 'USER', 'RETRY', 'TIME', 'RETURNCODE', 'COMMAND']
data = pd.read_csv('../data/slurm_wrapper_ce6.log', names=header_list, sep=' - ')

data

KeyError: 'Returncode'

In [ ]:
data = data.drop(columns='Retry')
data = data[data['RETURNCODE'] != 'returncode 0']
data

In [ ]:
data_user = data[data['RETURN'] == 'user 9204']
data_user


In [ ]:
data_user_sbatch = (data_user[data_user['COMMAND'].str.contains('sbatch')]
                   .reset_index()
                   .drop(columns='index'))
data_user_sbatch['Time'] = data_user_sbatch['Time'].map(lambda x: x.lstrip('time '))
data_user_sbatch['Returncode'] = data_user_sbatch['Returncode'].map(lambda x: x.lstrip('returncode '))
data_user_sbatch['Time'] = pd.to_numeric(data_user_sbatch['Time'])
data_user_sbatch = data_user_sbatch[(data_user_sbatch['Returncode'] == '1') & (data_user_sbatch['Time'] > 15)]
data_user_sbatch['SERVER'] = 'ce5'
data_user_sbatch

In [ ]:
data_two = pd.read_csv('../data/slurm_wrapper_ce5.log', names=header_list, sep=' - ')

data_two = data_two.drop(columns='Retry')
data_two = data_two[data_two['Returncode'] != 'returncode 0']

data_user_two = data_two[data_two['User'] == 'user 9204']

data_user_sbatch_two = (data_user_two[data_user_two['Command'].str.contains('sbatch')]
                   .reset_index()
                   .drop(columns='index'))
data_user_sbatch_two['Time'] = data_user_sbatch_two['Time'].map(lambda x: x.lstrip('time '))
data_user_sbatch_two['Returncode'] = data_user_sbatch_two['Returncode'].map(lambda x: x.lstrip('returncode '))
data_user_sbatch_two['Time'] = pd.to_numeric(data_user_sbatch_two['Time'])
data_user_sbatch_two = data_user_sbatch_two[(data_user_sbatch_two['Returncode'] == '1') & (data_user_sbatch_two['Time'] > 15)]
data_user_sbatch_two['SERVER'] = 'ce6'

In [ ]:
logs= data_user_sbatch.append(data_user_sbatch_two)
logs

In [ ]:
logs['Date'] = pd.to_datetime(logs['Date'])
logs.info()

In [ ]:
data_user_sbatch = data_user.sort_values('Date')
data_user_sbatch

data_user_sbatch = data_user_sbatch.reset_index()
data_user_sbatch = data_user_sbatch.drop(columns='index')
data_user_sbatch

data_user_sbatch['total_every_hour'] = (data_user_sbatch
                           .rolling('60min', on = 'Date')['Returncode']
                           .count())
